In [1]:
import numpy as np 
import pandas as pd

In [ ]:
from sklearn import tree

**Load data from csv files.**

In [6]:
df_A = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_A_feature.csv")
df_M = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_M_feature.csv")
df_S = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_S_feature.csv")

<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\F'
<>:3: SyntaxWarning: invalid escape sequence '\F'
<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:2: SyntaxWarning: invalid escape sequence '\F'
<>:3: SyntaxWarning: invalid escape sequence '\F'
C:\Users\sean\AppData\Local\Temp\ipykernel_22340\3918960565.py:1: SyntaxWarning: invalid escape sequence '\F'
  df_A = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_A_feature.csv")
C:\Users\sean\AppData\Local\Temp\ipykernel_22340\3918960565.py:2: SyntaxWarning: invalid escape sequence '\F'
  df_M = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 8803/ML-CSE-8803-Team-2/Classification/Stress/Datasets/F_Relax_M_feature.csv")
C:\Users\sean\AppData\Local\Temp\ipykernel_22340\3918960565.py:3: SyntaxWarning: invalid escape sequence '\F'
  df_S = pd.read_csv("C:/SEAN/GA Tech Academics/Grad\Fall 2024/ML CSE 88

Perform dimensionality reduction within each electrode's data. (There are 32 electrodes and 450 columns total, so 14 columns per electrode).

In [10]:
start_column_index = 1
for i in range(32):
    df_subset = df_A.iloc[:, start_column_index: start_column_index + 14]
    #print(df_subset.head())
    # Perform dimensionality reduction on df_subset
    
    start_column_index += 14

    0.psd_delta   0.psd_theta  0.psd_alpha   0.psd_beta  0.psd_gamma  \
0  14850.830823  14923.215094  7910.065985  6950.560915  3047.582095   
1  14838.280527  18650.065251  7921.853628  8087.625726  2855.607519   
2   7143.715839  16263.647994  6245.775968  4200.151132  2838.155117   
3   7143.570136  12413.634950  7677.763201  4200.428316  2764.924613   
4  19573.432251  18451.900407  7663.019348  6509.398324  2423.619039   

   0.theta_beta_ratio  0.theta_alpha_ratio  0.alpha_beta_ratio  \
0            2.147052             1.886611            1.138047   
1            2.306000             2.354255            0.979503   
2            3.872158             2.603944            1.487036   
3            2.955326             1.616830            1.827852   
4            2.834655             2.407915            1.177224   

   0.spectral_entropy_value  0.sample_entropy_value  0.delta_rel_power  \
0                  4.911304                2.639057           0.059237   
1                  4.7